# Patch Classification Training

### Part 1: Find all the patches in the small training set of Camelyon16

In [14]:
import repath.data.datasets.camelyon17 as camelyon17
from repath.preprocess.patching import GridPatchFinder
from repath.preprocess.patching import SlidesIndex
from repath.preprocess.tissue_detection import TissueDetectorGreyScale

# index all the patches for the camelyon16 dataset
train_data17 = camelyon17.training()
tissue_detector = TissueDetectorGreyScale()
patch_finder = GridPatchFinder(7, 0, 256, 256)
train_patches17 = SlidesIndex.index_dataset(train_data17, tissue_detector, patch_finder)
train_patches17.summary()

indexing patient_000_node_0.tif
indexing patient_000_node_1.tif
indexing patient_000_node_2.tif
indexing patient_000_node_3.tif
indexing patient_000_node_4.tif
indexing patient_001_node_0.tif
indexing patient_001_node_1.tif
indexing patient_001_node_2.tif
indexing patient_001_node_3.tif
indexing patient_001_node_4.tif
indexing patient_002_node_0.tif
indexing patient_002_node_1.tif
indexing patient_002_node_2.tif
indexing patient_002_node_3.tif
indexing patient_002_node_4.tif
indexing patient_003_node_0.tif
indexing patient_003_node_1.tif
indexing patient_003_node_2.tif
indexing patient_003_node_3.tif
indexing patient_003_node_4.tif
indexing patient_004_node_0.tif
indexing patient_004_node_1.tif
indexing patient_004_node_2.tif
indexing patient_004_node_3.tif
indexing patient_004_node_4.tif
indexing patient_005_node_0.tif
indexing patient_005_node_1.tif
indexing patient_005_node_2.tif
indexing patient_005_node_3.tif
indexing patient_005_node_4.tif
indexing patient_006_node_0.tif
indexing

KeyboardInterrupt: 

In [ ]:
from repath.preprocess.sampling import split_camelyon17
from repath.utils.paths import project_root

experiment_name = "example"
experiment_root = project_root() / "experiments" /  experiment_name

train17, valid17 = split_camelyon17(train_patches17, 0.7)
train17.save(experiment_root / "train17_index")
valid17.save(experiment_root / "valid17_index")
for sl in train17:
    print("train", sl.slide_idx)
    
for sl in valid17:
    print("valid", sl.slide_idx)

In [ ]:
project_root()

In [ ]:
from repath.preprocess.sampling import select_annotated

train17 = select_annotated(train17)
valid17 = select_annotated(valid17)

In [ ]:
import repath.data.datasets.camelyon16 as camelyon16
from repath.preprocess.patching import GridPatchFinder
from repath.preprocess.patching import SlidesIndex
from repath.preprocess.tissue_detection import TissueDetectorOTSU

# index all the patches for the camelyon16 dataset
train_data16 = camelyon16.training_small()
tissue_detector = TissueDetectorOTSU()
patch_finder = GridPatchFinder(7, 0, 256, 256)
train_patches16 = SlidesIndex.index_dataset(train_data16, tissue_detector, patch_finder)
train_patches16.summary()

In [ ]:
from repath.preprocess.sampling import split_camelyon16

train16, valid16 = split_camelyon16(train_patches16, 0.7)
train16.save(experiment_root / "train16_index")
valid16.save(experiment_root / "valid16_index")

In [ ]:
from repath.preprocess.sampling import balanced_sample

# do the train validate split
train_samples = balanced_sample([train16, train17], 2800)
valid_samples = balanced_sample([valid16, valid17], 1200)

import numpy as np
np.unique(train17.as_combined().patches_df.label)

In [ ]:
# save out all the patches
train_samples.save_patches(experiment_root / "training_patches")
valid_samples.save_patches(experiment_root / "validation_patches")

## Part 2: Train patch classifier

Annotation 1
metastases
Polygon
<Element 'Coordinates' at 0x7fa6bd7cf3b8>
[(69116.2969, 98458.2969), (68909.2969, 98291.5), (68690.7969, 98188.0), (68547.0, 98377.7969), (68679.2969, 98584.7969), (68754.0, 98803.3984), (68524.0, 98786.1016), (68460.7031, 99016.1016), (68558.5, 99240.3984), (68334.2031, 99188.7031), (68127.2031, 99056.3984), (68115.7031, 98803.3984), (67902.8984, 98918.3984), (67914.3984, 99148.3984), (68063.8984, 99332.5), (68178.8984, 99574.0), (68184.7031, 99809.7969), (67937.3984, 99884.6016), (67707.2969, 100023.0), (67580.7969, 100218.0), (67488.7969, 100471.0), (67396.7969, 100718.0), (67178.2031, 100828.0), (66930.8984, 100799.0), (66982.7031, 100563.0), (67224.2031, 100511.0), (67414.0, 100345.0), (67477.2969, 100109.0), (67299.0, 99959.2969), (67040.2031, 99947.7969), (66861.8984, 100126.0), (66631.8984, 100143.0), (66456.0, 100203.0), (66261.6016, 100348.0), (66016.5, 100270.0), (65887.1016, 100440.0), (65721.2031, 100582.0), (65655.8984, 100808.0), (65406.89